In [1]:
from sklearn.datasets import load_diabetes
from sklearn.metrics import r2_score
import numpy as np

In [3]:
x, y = load_diabetes(return_X_y = True)

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Using Stochastic Gradient Descent

In [23]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(penalty='l2', alpha=0.0001,max_iter=1000, learning_rate='constant', eta0=0.1)
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(r2)
print(reg.intercept_)
print(reg.coef_)

0.45966417458340336
[158.4558962]
[  42.85548936 -198.37232826  505.94132459  320.884399    -68.43551062
 -114.50503483 -214.51615585  147.0807582   373.66852751  107.09173472]


# Using Ridge Class

In [31]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=0.001, max_iter=1000, solver='sparse_cg')
# Solver: svd, cholesky, lsqr use OLS
# Solver: sparse_cg, sag, saga use GD
reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(r2)
print(reg.intercept_)
print(reg.coef_)

0.4534280297027061
151.34198054164818
[  38.4835076  -241.35179322  543.83518257  346.78283383 -827.70199038
  437.17375291  116.94974958  260.75926951  696.12922808   49.7434509 ]


# Creating our own class with GD

In [38]:
class RidgeRegressorGD:
    def __init__(self, epochs, learning_rate, alpha):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self, x_train, y_train):
        self.coef_ = np.ones(x_train.shape[1])
        self.intercept_ = 0
        x_train = np.insert(x_train, 0, 1, axis=1)
        beta = np.insert(self.coef_, 0, self.intercept_)
        for i in range(self.epochs):
            new_beta = np.insert(beta[1:], 0, 0)
            beta_der = np.dot(x_train.T, x_train).dot(beta)- np.dot(y_train.T, x_train) + self.alpha*new_beta
            beta = beta - self.learning_rate*beta_der
        self.coef_ = beta[1:]
        self.intercept_ = beta[0]
        
    def predict(self, x_test):
        return np.dot(x_test, self.coef_) + self.intercept_
        

In [40]:
rrgd = RidgeRegressorGD(1000, 0.005, 0.001)
rrgd.fit(x_train, y_train)
y_pred = rrgd.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(r2)
print(rrgd.intercept_)
print(rrgd.coef_)

0.4567905702813908
151.3084959365373
[  39.61965208 -233.61289214  546.05235734  338.82841788  -93.58555191
 -128.17957664 -216.7954418   147.82221494  406.11600122   77.89246013]
